In [ ]:
#Importing Libraries needed for NLP
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
#Importing Libraries needed for Tensorflow processing
import json
import numpy as np
import random
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import files
files.upload()

Saving intents.json to intents (2).json


{'intents (2).json': b'{\r\n        "intents": [\r\n                {\r\n                        "tag": "greeting",\r\n                        "patterns": [\r\n                                "Hi",\r\n                                "How are you",\r\n                                "Is anyone there?",\r\n                                "Hello",\r\n                                "Good day"\r\n                        ],\r\n                        "responses": [\r\n                                "Hello, thanks for visiting",\r\n                                "Good to see you again",\r\n                                "Hi there, how can I help?"\r\n                        ],\r\n                        "context_set": ""\r\n                },\r\n                {\r\n                        "tag": "goodbye",\r\n                        "patterns": [\r\n                                "Bye",\r\n                                "See you later",\r\n                                "Goodbye"\r\n 

In [ ]:
#Import our chat-bot intents file
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [ ]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [ ]:
words = []
classes = []
documents = []
ignore = ["?", "!", ".", ","]

In [ ]:
#Loop through each sentence in the intent's patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenize each and every word in the sentence
    w = nltk.word_tokenize(pattern)
    #add words to the words list
    words.extend(w)
    #add word(s) to the documents
    documents.append((w, intent['tag']))
    #add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [ ]:
#Performing stemming and lower each word as well as remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore]
words = sorted(set(words))

In [ ]:
#removing duplicates
classes = sorted(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

107 documents
32 classes ['about', 'advice', 'chatbot', 'compliment', 'connect', 'feedback', 'food', 'funfact', 'funny', 'future', 'games', 'goodbye', 'greeting', 'health', 'help', 'hours', 'jokes', 'learning', 'location', 'motivation', 'movies', 'music', 'news', 'pricing', 'quotes', 'restart', 'services', 'smalltalk', 'thanks', 'time', 'travel', 'weather']
156 unique stemmed words ["'m", "'s", 'a', 'about', 'account', 'address', 'advic', 'ai', 'an', 'ani', 'anyon', 'are', 'be', 'built', 'bye', 'can', 'chat', 'chatbot', 'clear', 'close', 'compliment', 'connect', 'cost', 'current', 'danger', 'day', 'destin', 'do', 'doe', 'down', 'eat', 'everyth', 'explain', 'fact', 'famou', 'favourit', 'feedback', 'feel', 'food', 'for', 'forecast', 'free', 'fun', 'funni', 'futur', 'game', 'get', 'give', 'go', 'good', 'goodby', 'guidanc', 'have', 'health', 'healthi', 'hello', 'help', 'here', 'hi', 'hour', 'how', 'hungri', 'i', 'idea', 'in', 'inspir', 'interest', 'is', 'issu', 'it', 'joke', 'know', 'later

In [ ]:
#Creating training data
training = []


#create an empty array for output
output_empty = [0]* len(classes)

#creating training sets, bag of words for each sentence
for doc in documents:
  #initialize bag of words
  bag =[]
  #list of tokenized words for the pattern
  pattern_words = doc[0]
  #Stemming each word
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  #create bag of words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  #output is '1' for current tag and '0' for the rest of other tags
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])


In [ ]:
# Shuffling features and turning it into np.array (shuffle the indices instead)
# We need to shuffle train_x and train_y together to maintain correspondence
random.shuffle(training)
training = np.array(training, dtype=object)

In [ ]:
# Convert the lists to NumPy arrays
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model2.keras')

Epoch 1/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0356 - loss: 3.5529
Epoch 2/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0678 - loss: 3.4096
Epoch 3/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0278 - loss: 3.4598
Epoch 4/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0649 - loss: 3.3840
Epoch 5/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1081 - loss: 3.3307
Epoch 6/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1027 - loss: 3.3139
Epoch 7/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1785 - loss: 3.2938
Epoch 8/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1472 - loss: 3.2260
Epoch 9/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1588 - loss: 3.1660
Epoch 10/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2012 - loss: 3.1414
Epoch 11/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1788 - loss: 3.1134
Epoch 12/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy:

In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [ ]:
from keras.models import load_model
model = load_model("/content/chatbot_model2.keras")

In [ ]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [ ]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

In [ ]:
def clean_up_sentence(sentence):
  #tokenize the pattern
  sentence_words = nltk.word_tokenize(sentence)
  #stem each word
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

#returning bag of words array : 0 or 1 for each  word in the bag that exists in the sentence
def bow(sentence, words):
  #tokenizing the pattern
  sentence_words = clean_up_sentence(sentence)

  #generating bag of words
  bag = [0]*len(words)
  for s in sentence_words:
    for i, w in enumerate(words):
      if w == s :
        bag[i] = 1
    bag = np.array(bag)
    return(bag)

In [ ]:
ERROR_THRESHOLD = 0.25

def classify(sentence, model):
  #generate probabilities from the model
  bag = bow(sentence, words)
  results = model.predict(np.array([bag]))
  #filter out predictions below a threshold
  results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
  #sort by strength of probability
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({"intent":classes[r[0]] , "probability": str(r[1])})
  #return tuple of intent and probability
  return return_list


In [ ]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])

In [ ]:
def chatbot_response(msg):
    ints = classify(msg, model)
    res = get_response(ints, intents)
    return res

In [ ]:
# Example chat
while True:
    inp = input("You: ")
    if inp.lower() == "quit":
        break
    print("Jarvis: ", chatbot_response(inp))

You: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Jarvis:  Hello, thanks for visiting
You: how are you doing?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Jarvis:  Hello, thanks for visiting
You: need a joke
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Jarvis:  Keep going! Great things take time. You've got this.
You: joku
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Jarvis:  Bye! Come back again soon.
You: jokes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Jarvis:  Why was the math book sad? It had too many problems.
You: tell me a story
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Jarvis:  If I had a dollar for every time I heard that… I’d still be a chatbot 😂
You: quit
